<a href="https://colab.research.google.com/github/MarioAuditore/TDA-for-Travelling-Salesman/blob/main/train_tsp_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Based on "The Transformer Network for the Traveling Salesman Problem"

Xavier Bresson, Thomas Laurent, Feb 2021<br>

Arxiv : https://arxiv.org/pdf/2103.03012.pdf<br>
Talk : https://ipam.wistia.com/medias/0jrweluovs<br>
Slides : https://t.co/ySxGiKtQL5<br>

This code trains the transformer network by reinforcement learning.<br>
Use the beam search code to test the trained network.


In [1]:
!git clone https://github.com/MarioAuditore/TDA-for-Travelling-Salesman.git

Cloning into 'TDA-for-Travelling-Salesman'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 95 (delta 9), reused 0 (delta 0), pack-reused 72
Receiving objects: 100% (95/95), 104.56 MiB | 26.23 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Updating files: 100% (35/35), done.


In [2]:
import os

os.chdir('/content/TDA-for-Travelling-Salesman')

In [3]:
!pip install 'pyconcorde @ git+https://github.com/jvkersch/pyconcorde'

  Cloning https://github.com/jvkersch/pyconcorde to /tmp/pip-install-ymfuinz6/pyconcorde_cfdb5e4ed805429bb905f7627b525036
  Running command git clone --filter=blob:none --quiet https://github.com/jvkersch/pyconcorde /tmp/pip-install-ymfuinz6/pyconcorde_cfdb5e4ed805429bb905f7627b525036
  Resolved https://github.com/jvkersch/pyconcorde to commit 8a6b193b79ebdf8f07e0b0635722b3b4edbc1560
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.0 MB/s eta 0:00:00
  Created wheel for pyconcorde: filename=pyconcorde-0.1.0-cp310-cp310-linux_x86_64.whl size=2576441 sha256=b5421c92031459d934a341e8f2065781add0132cabc3200e4a398f87d0377ad0
  Stored in directory: /tmp/pip-ephem-wheel-cache-1y9f4n55/wheels/be/9f/eb/e4f95e06e62f057a045679e4940e536d1b251d1ab4859c6dcc
Successfully built pyconcorde
  Attempting uninstall: tabu

In [4]:
!pip install giotto-ph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.4/526.4 kB 3.4 MB/s eta 0:00:00


In [5]:
# ================
# Libs
# ================

import torch
import torch.nn as nn
from tqdm import tqdm
import gph

# Models
from tsp_transformer.model import TSP_net, compute_tour_length


import time
import argparse
import os
import datetime


# visualization
%matplotlib inline
# from IPython.display import set_matplotlib_formats, clear_output
# set_matplotlib_formats('png2x','pdf')
import matplotlib.pyplot as plt
import numpy as np
# import pandas as pd


import networkx as nx
from scipy.spatial.distance import pdist, squareform
from concorde.tsp import TSPSolver # !pip install -e pyconcorde


import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [6]:
###################
# Hardware : CPU / GPU(s)
###################

if torch.backends.mps.is_available():
    gpu_id = '0'
    device = torch.device("mps")

elif torch.cuda.is_available():
    gpu_id = '0' # select a single GPU
    # gpu_id = '2,3' # select multiple GPUs
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    device = torch.device("cuda")
    print('GPU name: {:s}, gpu_id: {:s}'.format(torch.cuda.get_device_name(0),gpu_id))

else:
    device = torch.device("cpu")
    gpu_id = -1 # select CPU


print(device)

GPU name: Tesla T4, gpu_id: 0
cuda


In [7]:
# ================
# Hyper-parameters
# ================

class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

args = DotDict()
args.gpu_id = gpu_id

# TSP problem number of nodes
args.nb_nodes = 10 # TSP10
# args.nb_nodes = 20 # TSP20

# Transformer parameters
args.dim_emb = 128 # dimension of embeddings in transformer
args.dim_ff = 512 # dimension of feed forward layers
args.dim_input_nodes = 4 #2 # dimension of input features
args.nb_layers_encoder = 6
args.nb_layers_decoder = 2
args.nb_heads = 8

#
args.nb_epochs = 20 # number of epochs
args.batch_size = 128 # batch size
args.nb_batch_per_epoch = 250 # number of batches to generate on each epoch for training
args.nb_batch_eval = 100 # number of batches to generate on each epoch for evaluation
args.lr = 1e-4 # optimiser lr
args.tol = 1e-3 # model should perform better w.r.t tolerance to be updated
args.batchnorm = True  # if batchnorm=True  than batch norm is used
# args.batchnorm = False # if batchnorm=False than layer norm is used
args.max_len_PE = 1000

print(args)


{'gpu_id': '0', 'nb_nodes': 10, 'dim_emb': 128, 'dim_ff': 512, 'dim_input_nodes': 4, 'nb_layers_encoder': 6, 'nb_layers_decoder': 2, 'nb_heads': 8, 'nb_epochs': 20, 'batch_size': 128, 'nb_batch_per_epoch': 250, 'nb_batch_eval': 100, 'lr': 0.0001, 'tol': 0.001, 'batchnorm': True, 'max_len_PE': 1000}


# Training
## Setup

In [8]:
###################
# Instantiate a training network and a baseline network
###################
try:
    del model_train # remove existing model
    del model_baseline # remove existing model
except:
    pass

model_train = TSP_net(args.dim_input_nodes,
                      args.dim_emb,
                      args.dim_ff,
                      args.nb_layers_encoder,
                      args.nb_layers_decoder,
                      args.nb_heads,
                      args.max_len_PE,
                      batchnorm=args.batchnorm)

model_baseline = TSP_net(args.dim_input_nodes,
                         args.dim_emb,
                         args.dim_ff,
                         args.nb_layers_encoder,
                         args.nb_layers_decoder,
                         args.nb_heads,
                         args.max_len_PE,
                         batchnorm=args.batchnorm)

if torch.cuda.device_count() > 1:
    print(torch.cuda.device_count() + " cuda devices found, doing parallel training.")
    model_train = nn.DataParallel(model_train)
    model_baseline = nn.DataParallel(model_baseline)

optimizer = torch.optim.Adam(model_train.parameters(), lr = args.lr)

model_train = model_train.to(device)
model_baseline = model_baseline.to(device)
model_baseline.eval()

print(args); print('')

# Logs
os.system("mkdir logs")
time_stamp=datetime.datetime.now().strftime("%y-%m-%d--%H-%M-%S")
file_name = 'logs'+'/'+time_stamp + "-n{}".format(args.nb_nodes) + "-gpu{}".format(args.gpu_id) + ".txt"
file = open(file_name,"w",1)
file.write(time_stamp+'\n\n')
for arg in vars(args):
    file.write(arg)
    hyper_param_val="={}".format(getattr(args, arg))
    file.write(hyper_param_val)
    file.write('\n')
file.write('\n\n')
plot_performance_train = []
plot_performance_baseline = []
all_strings = []
epoch_ckpt = 0
tot_time_ckpt = 0


# # Uncomment these lines to re-start training with saved checkpoint
# ====================================================================
# checkpoint_file = "checkpoint/checkpoint_21-03-01--17-25-00-n50-gpu0.pkl"
# checkpoint = torch.load(checkpoint_file, map_location=device)
# epoch_ckpt = checkpoint['epoch'] + 1
# tot_time_ckpt = checkpoint['tot_time']
# plot_performance_train = checkpoint['plot_performance_train']
# plot_performance_baseline = checkpoint['plot_performance_baseline']
# model_baseline.load_state_dict(checkpoint['model_baseline'])
# model_train.load_state_dict(checkpoint['model_train'])
# optimizer.load_state_dict(checkpoint['optimizer'])
# print('Re-start training with saved checkpoint file={:s}\n  Checkpoint at epoch= {:d} and time={:.3f}min\n'.format(checkpoint_file,epoch_ckpt-1,tot_time_ckpt/60))
# del checkpoint
# ====================================================================



{'gpu_id': '0', 'nb_nodes': 10, 'dim_emb': 128, 'dim_ff': 512, 'dim_input_nodes': 4, 'nb_layers_encoder': 6, 'nb_layers_decoder': 2, 'nb_heads': 8, 'nb_epochs': 20, 'batch_size': 128, 'nb_batch_per_epoch': 250, 'nb_batch_eval': 100, 'lr': 0.0001, 'tol': 0.001, 'batchnorm': True, 'max_len_PE': 1000}



## Test
### Best solution

In [9]:
from joblib import Parallel, delayed

create_checkpoint = False
test_size = 1000
n_nodes = args.nb_nodes

if create_checkpoint:
    x = torch.rand(test_size, n_nodes, args.dim_input_nodes, device='cpu')

    def best_path(points):
        solution = TSPSolver.from_data(points[:, 0], points[:, 1], norm='GEO').solve(verbose=False)
        return compute_tour_length(points[None, :], solution.tour[None, :]).to('cpu')

    lengths = Parallel(n_jobs=-1)(delayed(best_path)(points) for points in x)

    best_length_mean = torch.tensor(lengths).mean()

    data_dir = os.path.join("data")
    torch.save({ 'x': x, 'mean': best_length_mean}, '{}.pkl'.format(data_dir + f"/{test_size}tsp{n_nodes}"))

### TSP tasks as validation checkpoints

In [14]:
###################
# Small test set for quick algorithm comparison
# Note : this can be removed
###################

checkpoint = None

if args.nb_nodes==10 : checkpoint = torch.load("data/1000tsp10.pkl")
if args.nb_nodes==20 : checkpoint = torch.load("data/1000tsp20.pkl")

if checkpoint is not None:
    x_test_tsp = checkpoint['x']
    x_test_len = checkpoint['mean']
    n = x_test_tsp.size(1)
    print(f'nb of nodes : {n} | mean length: {x_test_len}')
else:
    x_test_tsp = torch.rand(test_size, args.nb_nodes, args.dim_input_nodes, device='cpu')
    x_test_len = 1e-7
    n = x_test_tsp.size(1)
    print('nb of nodes :',n)

nb of nodes : 10 | mean length: 2.882951259613037


## TDA

In [11]:
def calculate_homology(points):
    # Calculate homology using ripser_parallel
    output = gph.ripser_parallel(points, return_generators=True)

    # Initialize arrays for H0_birth, H0_death, H1_birth, and H1_death
    num_points = points.shape[0]
    # H0_birth, H0_death = np.zeros(num_points, dtype=int), np.zeros(num_points, dtype=int)
    H1_birth, H1_death = np.zeros(num_points, dtype=int), np.zeros(num_points, dtype=int)

    # Iterate over finite generators for H0
    # for i in range(len(output['gens'][0])):
    #     birth_vertex = output['gens'][0][i][0]
    #     death_vertices = output['gens'][0][i][1:]

    #     H0_birth[birth_vertex] = 1
    #     H0_death[death_vertices] = 1

    # Iterate over infinite generators for H0
    # for i in range(len(output['gens'][2])):
    #     birth_vertex = output['gens'][2][i]

    #     H0_birth[birth_vertex] = 1

    # Iterate over finite generators for H1
    for i in range(len(output['gens'][1][0][:2])):
        birth_vertices = output['gens'][1][0][i][:2]
        death_vertices = output['gens'][1][0][i][2:]

        H1_birth[birth_vertices] = 1
        H1_death[death_vertices] = 1

    # Iterate over infinite generators for H1
    for i in range(len(output['gens'][3])):
        birth_vertices = output['gens'][3][i]
        if birth_vertices.size > 0:
            H1_birth[birth_vertices] = 1

    return np.column_stack((points[:,0], points[:,1], H1_birth, H1_death))

# Example usage:
# points = np.array([[0, 0], [1, 1], [2, 2], [3, 3]])
# H0_birth, H0_death, H1_birth, H1_death = calculate_homology(points)

## Training loop

In [17]:
###### ==================
# Main training loop
# ==================
from joblib import Parallel, delayed


start_training_time = time.time()

for epoch in tqdm(range(0, args.nb_epochs)):

    # re-start training with saved checkpoint
    # epoch += epoch_ckpt

    # -------------------------
    # Train model for one epoch
    # -------------------------
    start = time.time()
    model_train.train()

    for step in range(1, args.nb_batch_per_epoch + 1):

        # generate a batch of random TSP instances
        x = torch.rand(args.batch_size, args.nb_nodes, args.dim_input_nodes) # size(x)=(batch_size, nb_nodes, dim_input_nodes)

        # generate topological features
        # new_f = ...
        # data = x # np.concat(x, new_f)
        data = torch.tensor(Parallel(n_jobs=-1)(delayed(calculate_homology)(i) for i in x), dtype=torch.float32)

        # compute tours for model
        tour_train, sumLogProbOfActions = model_train(data.to(device), deterministic=False) # size(tour_train)=(batch_size, nb_nodes), size(sumLogProbOfActions)=(batch_size)

        # compute tours for baseline
        with torch.no_grad():
            tour_baseline, _ = model_baseline(data.to(device), deterministic=True)

        # get the lengths of the tours
        x = x.to(device)
        L_train = compute_tour_length(x, tour_train) # size(L_train)=(batch_size)
        L_baseline = compute_tour_length(x, tour_baseline) # size(L_baseline)=(batch_size)

        # backprop
        loss = torch.mean((L_train - L_baseline) * sumLogProbOfActions )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    time_one_epoch = time.time()-start
    time_tot = time.time()-start_training_time + tot_time_ckpt


    # -----------------
    # Evaluate train model and baseline on 10k random TSP instances
    # -----------------
    model_train.eval()
    mean_tour_length_train = 0
    mean_tour_length_baseline = 0

    for step in range(0, args.nb_batch_eval):
        # generate a batch of random tsp instances
        x = torch.rand(args.batch_size, args.nb_nodes, args.dim_input_nodes)

        # generate topological features
        # new_f = ...
        # data = x # np.concat(x, new_f)
        torch.tensor(Parallel(n_jobs=-1)(delayed(calculate_homology)(i) for i in x), dtype=torch.float32)

        # compute tour for model and baseline
        with torch.no_grad():
            tour_train, _ = model_train(data.to(device), deterministic=True)
            tour_baseline, _ = model_baseline(data.to(device), deterministic=True)

        # get the lengths of the tours
        L_train = compute_tour_length(x.to(device), tour_train)
        L_baseline = compute_tour_length(x.to(device), tour_baseline)

        # L_tr and L_bl are tensors of shape (batch_size,). Compute the mean tour length
        mean_tour_length_train += L_train.mean().item()
        mean_tour_length_baseline += L_baseline.mean().item()

    mean_tour_length_train =  mean_tour_length_train / args.nb_batch_eval
    mean_tour_length_baseline =  mean_tour_length_baseline / args.nb_batch_eval

    # evaluate train model and baseline and update if train model is better
    update_baseline = mean_tour_length_train + args.tol < mean_tour_length_baseline
    if update_baseline:
        model_baseline.load_state_dict(model_train.state_dict())

    # For new baseline compute TSPs for small test set
    with torch.no_grad():
        data = torch.tensor(Parallel(n_jobs=-1)(delayed(calculate_homology)(i) for i in x_test_tsp), dtype=torch.float32)
        tour_baseline, _ = model_baseline(data.to(device), deterministic=True)
    mean_tour_length_test = compute_tour_length(x_test_tsp, tour_baseline.to('cpu')).mean().item()

    # For checkpoint
    plot_performance_train.append([(epoch+1), mean_tour_length_train])
    plot_performance_baseline.append([(epoch+1), mean_tour_length_baseline])

    # Compute optimality gap
    gap_train = mean_tour_length_test / x_test_len - 1.0


    # Print and save in txt file
    mystring_min = 'Epoch: {:d}, epoch time: {:.3f}min, tot time: {:.3f}day, L_train: {:.3f}, L_base: {:.3f}, L_test: {:.3f}, gap_train(%): {:.3f}, update: {}'.format(
        epoch, time_one_epoch/60, time_tot/86400, mean_tour_length_train, mean_tour_length_baseline, mean_tour_length_test, 100*gap_train, update_baseline)
    print(mystring_min) # Comment if plot display

    # all_strings.append(mystring_min) # Uncomment if plot display
    # for string in all_strings:
    #     print(string)

    # Saving checkpoint
    checkpoint_dir = os.path.join("checkpoint")
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    torch.save({
        'epoch': epoch,
        'time': time_one_epoch,
        'tot_time': time_tot,
        'loss': loss.item(),
        'TSP_length': [torch.mean(L_train).item(), torch.mean(L_baseline).item(), mean_tour_length_test],
        'plot_performance_train': plot_performance_train,
        'plot_performance_baseline': plot_performance_baseline,
        'mean_tour_length_test': mean_tour_length_test,
        'model_baseline': model_baseline.state_dict(),
        'model_train': model_train.state_dict(),
        'optimizer': optimizer.state_dict(),
        }, '{}.pkl'.format(checkpoint_dir + "/checkpoint_" + time_stamp + "-n{}".format(args.nb_nodes) + "-gpu{}".format(args.gpu_id)))



  5%|▌         | 1/20 [01:18<24:46, 78.22s/it]

Epoch: 0, epoch time: 0.968min, tot time: 0.001day, L_train: 7.782, L_base: 7.777, L_test: 3.043, gap_train(%): 5.562, update: False


  5%|▌         | 1/20 [01:55<36:25, 115.03s/it]


KeyboardInterrupt: 

## Final check on test data

In [19]:
# generate topological features
# new_f = ...
data = x_test_tsp # np.concat(x_test_tsp, new_f)

with torch.no_grad():
    data = torch.tensor(Parallel(n_jobs=-1)(delayed(calculate_homology)(i) for i in x_test_tsp), dtype=torch.float32)
    tour_baseline, _ = model_baseline(data.to(device), deterministic=True)
    print(compute_tour_length(x_test_tsp, tour_baseline.to('cpu')).mean(), x_test_len)

tensor(3.0433) tensor(2.8830)


# Visualisation
Graphs should be fixed

In [ ]:
from scipy.spatial import distance_matrix


idx = 0
points = np.array(x_test_tsp[idx].to('cpu'))

dist_mtx = distance_matrix(points, points)
model_tour = np.array(tour_baseline[idx].to('cpu'))

def create_graph(tour, dist_mtx):
    mask = np.zeros_like(dist_mtx)

    for i in range(-1, len(tour) - 1):
        point_a = tour[i]
        point_b = tour[i + 1]
        mask[point_a, point_b] = 1
    return nx.from_numpy_matrix(mask * dist_mtx)

G = create_graph(model_tour, dist_mtx)
pos = nx.spring_layout(G, seed=47)  # Seed layout for reproducibility
nx.draw(G)
plt.show()

In [ ]:
concorde_tour = TSPSolver.from_data(points[:, 0], points[:, 1], norm='GEO').solve(verbose=False).tour

In [ ]:
G = create_graph(concorde_tour, dist_mtx)
pos = nx.spring_layout(G, seed=47)  # Seed layout for reproducibility
nx.draw(G)
plt.show()